
# 🌿 Graves Greenery — Colab SQL Template (DuckDB + `%%sql` + GitHub CSVs)

This notebook gives you a **serverless SQL sandbox** for the Graves Greenery project:
- Loads CSVs from your **GitHub repo** (public)
- Creates tables in a **DuckDB** database (file-backed, persisted in Colab)
- Lets you query with **`%%sql` magic** (PrettyTable-style output)
- Optional **`%%mysql` magic** to write MySQL syntax and run via DuckDB using SQLGlot


## 1) Install packages

In [ ]:
!pip -q install duckdb ipython-sql pandas duckdb-engine sqlglot

## 2) Imports & load SQL magic

In [ ]:
import os, re, glob, getpass, subprocess, textwrap, pandas as pd, duckdb
from pathlib import Path

# Load the ipython-sql extension
%load_ext sql

## 3) Configuration — repo & DB paths (pre-filled)

In [ ]:
# === EDIT ONLY IF NEEDED ===
GITHUB_USER   = "danielgraves"
GITHUB_REPO   = "Graves_Greenery_Analysis"
GITHUB_BRANCH = "main"
SPARSE_PATHS  = ["data"]              # not used for public clone, but kept for reference
IS_PRIVATE    = False                 # public per your setup

# CSVs live in /data within the repo
CSV_GLOB      = "data/**/*.csv"       # recursive search under data/

# Persistent DuckDB database file (saved in Colab environment)
DB_PATH       = "/content/graves_greenery.duckdb"

# Local repo directory in Colab
REPO_DIR      = f"/content/{GITHUB_REPO}"

# Namespacing control: include parent folder names in table names to avoid collisions
INCLUDE_PARENT_IN_TABLE = False  # True -> tables like data_orders; False -> orders

## 4) Start DuckDB (file-backed) and connect `%%sql`

In [ ]:
# Ensure parent folder exists
Path(DB_PATH).parent.mkdir(parents=True, exist_ok=True)

# Initialize or connect to the DB
_ = duckdb.connect(DB_PATH)
_.close()

# Connect ipython-sql to the same DB file (shared with %%sql)
%sql duckdb:///{DB_PATH}
print("Connected to DuckDB:", DB_PATH)

## 5) Clone (or Pull) your GitHub repo (public)

In [ ]:
def clone_or_pull_repo(user, repo, branch, dest):
    if os.path.exists(dest):
        print(f"Repo exists at {dest}. Pulling latest...")
        subprocess.run(f"git -C {dest} pull --ff-only", shell=True, check=True)
        return

    cmd = f"git clone --depth 1 --branch {branch} https://github.com/{user}/{repo}.git {dest}"
    print(cmd)
    subprocess.run(cmd, shell=True, check=True)

clone_or_pull_repo(GITHUB_USER, GITHUB_REPO, GITHUB_BRANCH, REPO_DIR)
print("Repo ready at:", REPO_DIR)

## 6) Load CSVs into DuckDB tables (auto schema inference)

In [ ]:
con = duckdb.connect(DB_PATH)

def slugify_table_name(path, include_parent=False):
    path = Path(path)
    stem = path.stem.lower()
    stem = re.sub(r"[^a-z0-9_]+", "_", stem)
    stem = re.sub(r"_+", "_", stem).strip("_")
    if include_parent and path.parent != path.parent.parent:
        parent = re.sub(r"[^a-z0-9_]+", "_", path.parent.name.lower()).strip("_")
        name = f"{parent}_{stem}"
    else:
        name = stem
    if re.match(r"^\\d", name):
        name = "t_" + name
    return name

def load_csvs_as_tables(repo_dir, csv_glob, include_parent=False):
    csvs = glob.glob(os.path.join(repo_dir, csv_glob), recursive=True)
    loaded = []
    for f in csvs:
        tbl = slugify_table_name(f, include_parent=include_parent)
        # Use read_csv_auto for robust type inference; ignore bad rows to keep loading resilient
        con.execute(f"""
            CREATE OR REPLACE TABLE "{tbl}" AS
            SELECT * FROM read_csv_auto(?, header=True, sample_size=-1, ignore_errors=True);
        """, [f])
        loaded.append((tbl, f))
    return loaded

loaded = load_csvs_as_tables(REPO_DIR, CSV_GLOB, INCLUDE_PARENT_IN_TABLE)
print(f"Loaded {len(loaded)} CSVs into DuckDB tables.")
loaded[:10]  # preview first 10

## 7) Verify: list available tables

In [ ]:
%%sql
SELECT table_name
FROM duckdb_tables()
WHERE NOT internal
ORDER BY table_name;

## 8) (Optional) `%%mysql` magic — write MySQL, run on DuckDB via SQLGlot

In [ ]:
from sqlglot import transpile
from IPython.core.magic import register_cell_magic

@register_cell_magic
def mysql(line, cell):
    """
    Usage:
    %%mysql
    SELECT DATE_FORMAT(CURRENT_DATE(), '%Y-%m-%d') AS today;
    """
    # Translate MySQL -> DuckDB
    [duckdb_sql] = transpile(cell, read="mysql", write="duckdb")
    # Minor compatibility tweaks (extend as needed)
    duckdb_sql = (duckdb_sql
                  .replace("IFNULL", "COALESCE")
                  .replace("NOW()", "CURRENT_TIMESTAMP"))
    print("Translated to DuckDB SQL:\n", duckdb_sql, "\n", flush=True)
    # Execute through ipython-sql so we get PrettyTable rendering
    return get_ipython().run_cell_magic('sql', '', duckdb_sql)

print("Custom %%mysql magic is ready. Try it in the next cell.")

## 9) Example queries — Graves Greenery Starter

In [ ]:
# ⚠️ Edit table names below if your CSV filenames differ.
# Common expected tables: plants, categories, customers, orders, order_items

# Top sellers by revenue
_ = get_ipython().run_cell_magic('sql', '', textwrap.dedent("""
SELECT p.name AS plant,
       SUM(oi.quantity)                        AS units_sold,
       SUM(oi.quantity * oi.unit_price)        AS revenue
FROM order_items oi
JOIN plants p ON p.plant_id = oi.plant_id
GROUP BY 1
ORDER BY revenue DESC
LIMIT 10;
"""))

# Repeat purchase rate (simple version)
_ = get_ipython().run_cell_magic('sql', '', textwrap.dedent("""
WITH per_customer AS (
  SELECT customer_id, COUNT(DISTINCT order_id) AS orders_n
  FROM orders
  GROUP BY customer_id
)
SELECT
  SUM(CASE WHEN orders_n >= 2 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS repeat_rate
FROM per_customer;
"""))

# Monthly revenue trend
_ = get_ipython().run_cell_magic('sql', '', textwrap.dedent("""
SELECT
  DATE_TRUNC('month', order_date)::DATE AS month,
  SUM(order_total) AS revenue
FROM orders
GROUP BY 1
ORDER BY 1;
"""))

## 10) Refresh data (git pull + reload CSVs)

In [ ]:
# Pull latest changes (re-run this cell when repo updates)
subprocess.run(f"git -C {REPO_DIR} pull --ff-only", shell=True, check=True)

# Rebuild tables
loaded = load_csvs_as_tables(REPO_DIR, CSV_GLOB, INCLUDE_PARENT_IN_TABLE)
print(f"Reloaded {len(loaded)} CSVs into DuckDB tables.")

## 11) Snapshot / export

In [ ]:
# Option A: The DuckDB file at DB_PATH is your snapshot already.
print("DuckDB snapshot at:", DB_PATH)

# Option B: Export a query to CSV (example)
export_path = "/content/top_plants.csv"
_ = get_ipython().run_cell_magic('sql', '', textwrap.dedent(f"""
COPY (
  SELECT p.name AS plant,
         SUM(oi.quantity)                 AS units_sold,
         SUM(oi.quantity * oi.unit_price) AS revenue
  FROM order_items oi
  JOIN plants p ON p.plant_id = oi.plant_id
  GROUP BY 1
  ORDER BY revenue DESC
  LIMIT 50
) TO '{export_path}' WITH (HEADER, DELIMITER ',');
"""))
print("Exported CSV:", export_path)
